In [6]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.utils

In [7]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

In [28]:
df_green = spark.read \
    .option("header","true") \
    .parquet("data/pq/green/*/*")

In [29]:
df_yellow = spark.read \
    .option("header","true") \
    .parquet("data/pq/yellow/*/*")

In [30]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime') \
                    .withColumn('service_type', lit('green'))

df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
                    .withColumn('service_type', lit('yellow'))

In [31]:
common_columns = []

for col in set(df_green.columns):
    if col in set(df_yellow.columns):
        common_columns.append(col)

common_columns

['DOLocationID',
 'RatecodeID',
 'VendorID',
 'extra',
 'passenger_count',
 'pickup_datetime',
 'store_and_fwd_flag',
 'total_amount',
 'mta_tax',
 'tip_amount',
 'PULocationID',
 'trip_distance',
 'fare_amount',
 'dropoff_datetime',
 'congestion_surcharge',
 'tolls_amount',
 'service_type',
 'payment_type',
 'improvement_surcharge']

In [32]:
df_green = df_green.select(common_columns)
df_yellow = df_yellow.select(common_columns)

In [36]:
trip_data = df_green.unionAll(df_yellow)

In [37]:
trip_data.show()

+------------+----------+--------+-----+---------------+-------------------+------------------+------------+-------+----------+------------+-------------+-----------+-------------------+--------------------+------------+------------+------------+---------------------+
|DOLocationID|RatecodeID|VendorID|extra|passenger_count|    pickup_datetime|store_and_fwd_flag|total_amount|mta_tax|tip_amount|PULocationID|trip_distance|fare_amount|   dropoff_datetime|congestion_surcharge|tolls_amount|service_type|payment_type|improvement_surcharge|
+------------+----------+--------+-----+---------------+-------------------+------------------+------------+-------+----------+------------+-------------+-----------+-------------------+--------------------+------------+------------+------------+---------------------+
|         173|         1|       2|  1.0|              1|2020-01-16 19:49:27|                 N|        14.3|    0.5|       0.0|         260|         2.59|       12.5|2020-01-16 20:06:44|       